# Notebook Lecture 1: Introduction, Digital Control Systems
© 2025 ETH Zurich, Niclas Scheuer; Institute for Dynamic Systems and Control; Prof. Emilio Frazzoli

This interactive notebook covers the basics of discrete-time control and aliasing.

Authors:
- Niclas Scheuer; nscheuer@ethz.ch

## Import the packages

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from scipy.optimize import curve_fit
from IPython.display import display

# Example 1: Zero-Order Hold

The zero-order hold is perhaps the simplest method to turn a _discrete-time_ (DT) signal into a _continuous-time_ (CT) signal. It does this by taking the value of the input at time $$t_0 = kT$$ and holding this output for the whole sampling period $$t \in [kT, (k+1)T)$$



In [ ]:
def zero_order_hold_plot(fs):
    T = 2  # Total time in seconds
    t_cont = np.linspace(0, T, 1000)  # Continuous time axis
    signal_cont = np.sin(2 * np.pi * 1 * t_cont)  # Continuous sine wave
    
    t_samples = np.arange(0, T, 1/fs)  # Discrete sample times
    signal_samples = np.sin(2 * np.pi * 1 * t_samples)  # Sampled signal
    
    # Create ZOH signal by repeating each sample value until the next sample
    t_zoh = np.repeat(t_samples, 2)
    signal_zoh = np.repeat(signal_samples, 2)
    
    # Adjust last element to match the original timeline
    t_zoh = np.append(t_zoh, T)
    signal_zoh = np.append(signal_zoh, signal_zoh[-1])
    
    plt.figure(figsize=(8, 4))
    plt.plot(t_cont, signal_cont, 'b', label='Original Signal')  # Continuous signal
    plt.step(t_zoh, signal_zoh, 'r', where='post', label='Zero-Order Hold')  # ZOH signal
    plt.scatter(t_samples, signal_samples, color='black', zorder=3, label='Samples')  # Sample points
    
    plt.xlabel('Time (s)')
    plt.ylabel('Amplitude')
    plt.title(f'Zero-Order Hold (Sampling Frequency = {fs} Hz)')
    plt.legend()
    plt.grid(True)
    plt.show()

# Create slider for sampling frequency
fs_slider = widgets.IntSlider(min=1, max=30, step=1, value=5, description='Fs (Hz)')
interactive_plot = widgets.interactive_output(zero_order_hold_plot, {'fs': fs_slider})
display(fs_slider, interactive_plot)


IntSlider(value=5, description='Fs (Hz)', max=30, min=1)

Output()

# Example 2: Nyquist-Shannon Sampling Theorem

If we don't sample a signal often enough, we lose important details, making it hard to reconstruct the original signal.

## Sinuisoidal Signals and Sampling
Since any signal can be broken down into sine waves using the Fourier transform, we use a basic sinusoidal signal to explain the Nyquist-Shannon Sampling Theorem.

## Nyquist-Shannon Sampling Theorem
A sampled signal can be perfectly reconstructed only if the sampling rate is at least twice the highest frequency present in the signal.

$$
f_s \geq 2f_{max}
$$

where:
- $f_s$ is the **sampling frequency**
- $f_{max}$ is the **highest frequency component** in the signal





In [3]:
# Function to generate a sinusoidal signal
def sinusoid(t, f):
    return np.sin(2 * np.pi * f * t)

# Function to fit a sinusoid to sampled points using the lowest frequency possible
def fit_sinusoid(t_samples, y_samples, f_s):
    def model(t, A, f_fit, phase):
        return A * np.sin(2 * np.pi * f_fit * t + phase)
    
    # Nyquist folding to find the aliased frequency
    f_alias = np.abs((f_samples := np.fft.fftfreq(len(t_samples), d=(t_samples[1] - t_samples[0])))[np.argmax(np.abs(np.fft.fft(y_samples)))])
    f_guess = f_alias if f_alias < f_s / 2 else f_s - f_alias
    
    params, _ = curve_fit(model, t_samples, y_samples, p0=[1, f_guess, 0])
    return lambda t: model(t, *params)

# Interactive function to update the plot
def nyquist_plot(f_max, f_s):
    T = 1  # Total duration (1 second)
    t_cont = np.linspace(0, T, 1000)  # High-resolution time for continuous signal
    y_cont = sinusoid(t_cont, f_max)  # Continuous signal

    # Sampling points
    t_samples = np.arange(0, T, 1/f_s)  # Sampled times
    y_samples = sinusoid(t_samples, f_max)  # Sampled values

    # Fit a sinusoid to sampled points
    fitted_sinusoid = fit_sinusoid(t_samples, y_samples, f_s)

    # Generate reconstructed signal
    y_reconstructed = fitted_sinusoid(t_cont)

    # Plot the results
    plt.figure(figsize=(8, 4))
    plt.plot(t_cont, y_cont, 'b', label='Original Signal')  # Continuous signal
    plt.scatter(t_samples, y_samples, color='red', zorder=3, label='Sampled Points')  # Sampled points
    plt.plot(t_cont, y_reconstructed, 'g--', label='Reconstructed Signal')  # Fitted sinusoid
    
    # Nyquist Criterion Check
    nyquist_condition = f_s >= 2 * f_max
    condition_text = "✓ Nyquist Theorem Fulfilled" if nyquist_condition else "✗ Undersampling (Aliasing Can Occur!)"
    plt.title(f'Nyquist-Shannon Sampling Theorem\n{condition_text}', fontsize=12, color='green' if nyquist_condition else 'red')

    plt.xlabel('Time (s)')
    plt.ylabel('Amplitude')
    plt.legend()
    plt.grid(True)
    plt.show()

# Create interactive sliders
f_max_slider = widgets.FloatSlider(min=1, max=10, step=0.5, value=3, description='f_max (Hz)')
f_s_slider = widgets.FloatSlider(min=2, max=20, step=0.5, value=6, description='f_s (Hz)')

# Display interactive plot
interactive_plot = widgets.interactive_output(nyquist_plot, {'f_max': f_max_slider, 'f_s': f_s_slider})
display(f_max_slider, f_s_slider, interactive_plot)


FloatSlider(value=3.0, description='f_max (Hz)', max=10.0, min=1.0, step=0.5)

FloatSlider(value=6.0, description='f_s (Hz)', max=20.0, min=2.0, step=0.5)

Output()

In [4]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from scipy.fftpack import fft, fftfreq
import ipywidgets as widgets
from IPython.display import display

# Global storage for the original signal
global_t_cont = np.linspace(0, 1, 1000)  # High-resolution time
global_freq_options = np.array([1, 2, 5, 10, 20])  # Available frequency components
global_num_components = 2  # Default number of harmonics
global_f_s = 20  # Default sampling frequency

def generate_signal(num_components):
    global global_freqs, global_amps, global_phases, global_y_cont
    global_freqs = global_freq_options[:num_components]
    global_amps = np.ones(len(global_freqs))
    global_phases = np.random.uniform(0, 2*np.pi, len(global_freqs))
    global_y_cont = sum(A * np.sin(2 * np.pi * f * global_t_cont + p) for A, f, p in zip(global_amps, global_freqs, global_phases))

generate_signal(global_num_components)

# Function to fit multiple sinusoids to sampled points
def fit_sinusoids(t_samples, y_samples, freqs):
    def model(t, *params):
        A = params[:len(freqs)]
        P = params[len(freqs):]
        return sum(A[i] * np.sin(2 * np.pi * freqs[i] * t + P[i]) for i in range(len(freqs)))
    
    p0 = [1] * len(freqs) + [0] * len(freqs)  # Initial guess: Amplitudes = 1, Phases = 0
    params, _ = curve_fit(model, t_samples, y_samples, p0=p0)
    return lambda t: model(t, *params)

# Function to plot Nyquist sampling with complex signals
def nyquist_plot():
    T = 1  # Total duration (1 second)
    num_samples = int(T * global_f_s)  # Adjust number of samples dynamically
    t_samples = np.linspace(0, T, num_samples, endpoint=False)
    y_samples = sum(A * np.sin(2 * np.pi * f * t_samples + p) for A, f, p in zip(global_amps, global_freqs, global_phases))

    # Reconstructed signal
    fitted_sinusoids = fit_sinusoids(t_samples, y_samples, global_freqs)
    y_reconstructed = fitted_sinusoids(global_t_cont)

    # Nyquist Criterion Check
    max_freq = max(global_freqs)
    nyquist_condition = global_f_s >= 2 * max_freq
    condition_text = "\n\n✓ Nyquist Theorem Fulfilled" if nyquist_condition else "\n\n✗ Undersampling (Aliasing Occurs!)"

    # Plot time domain
    plt.figure(figsize=(10, 6))
    plt.subplot(2, 1, 1)
    plt.title(f'Nyquist-Shannon Sampling Theorem {condition_text}', fontsize=12, color='green' if nyquist_condition else 'red')
    plt.plot(global_t_cont, global_y_cont, 'b', label='Original Signal')
    plt.scatter(t_samples, y_samples, color='red', zorder=3, label='Sampled Points')
    plt.plot(global_t_cont, y_reconstructed, 'g--', label='Reconstructed Signal')
    plt.xlabel('Time (s)')
    plt.ylabel('Amplitude')
    plt.legend()
    plt.grid(True)

    # Fourier Transform Analysis
    N = 4096  # Increase FFT size for better resolution
    freqs_cont = fftfreq(N, d=(global_t_cont[1] - global_t_cont[0]))
    spectrum_cont = np.abs(fft(global_y_cont, n=N))
    spectrum_cont /= np.max(spectrum_cont)  # Normalize original spectrum
    
    N_samples = 4096  # Ensure FFT resolution matches
    freqs_samples = fftfreq(N_samples, d=(t_samples[1] - t_samples[0]))
    spectrum_samples = np.abs(fft(y_samples, n=N_samples))
    spectrum_samples /= np.max(spectrum_samples)  # Normalize sampled spectrum
    
    plt.subplot(2, 1, 2)
    plt.plot(freqs_cont[:N//2], spectrum_cont[:N//2], 'b', label='Original Spectrum')
    plt.plot(freqs_samples[:N_samples//2], spectrum_samples[:N_samples//2], 'r--', label='Sampled Spectrum')
    plt.xlim(0, 40)  # Limit frequency axis to 40 Hz
    plt.xlabel('Frequency (Hz)')
    plt.ylabel('Normalized Magnitude')
    plt.legend()
    plt.grid(True)

    # Annotate expected peaks
    for f in global_freqs:
        plt.axvline(f, color='gray', linestyle='dashed', alpha=0.6)
    
    plt.tight_layout()
    plt.show()

# Interactive controls
num_components_slider = widgets.IntSlider(min=1, max=len(global_freq_options), step=1, value=global_num_components, description='Harmonics')
f_s_slider = widgets.FloatSlider(min=5, max=50, step=1, value=global_f_s, description='f_s (Hz)')

def update_plot(num_components, f_s):
    global global_f_s
    if num_components != len(global_freqs):
        generate_signal(num_components)
    global_f_s = f_s
    nyquist_plot()

interactive_plot = widgets.interactive_output(update_plot, {'num_components': num_components_slider, 'f_s': f_s_slider})
display(num_components_slider, f_s_slider, interactive_plot)


IntSlider(value=2, description='Harmonics', max=5, min=1)

FloatSlider(value=20.0, description='f_s (Hz)', max=50.0, min=5.0, step=1.0)

Output()

# Exercise 3: Inverted Pendulum Control



In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output
from scipy import signal  # Importing the signal module from scipy

# Define sinusoidal signal (Reference or Noise)
def sinusoidal_signal(t, amplitude, frequency):
    return amplitude * np.sin(2 * np.pi * frequency * t)

# Apply Anti-Aliasing Filter (Low-pass filter)
def apply_anti_aliasing_filter(input_signal, cutoff_frequency, sampling_rate=1000):
    # Design a low-pass filter using scipy.signal
    nyquist = 0.5 * sampling_rate
    normal_cutoff = cutoff_frequency / nyquist
    b, a = signal.butter(1, normal_cutoff, btype='low', analog=False)
    filtered_signal = signal.filtfilt(b, a, input_signal)  # Using the input_signal
    return filtered_signal

def pid_controller(error, p_gain, i_gain, d_gain, dt=1.0):
    integral = 0
    derivative = 0
    prev_error = 0

    control_signal = []

    for i in range(len(error)):
        P_term = p_gain * error[i]

        # Integral error
        integral += error[i] * dt
        I_term = i_gain * integral

        # Derivative error
        derivative = (error[i] - prev_error) / dt
        D_term = d_gain * derivative

        # PID control signal
        u = P_term + I_term + D_term

        control_signal.append(u)

        prev_error = error[i]

    return np.array(control_signal)

def inverted_pendulum_transfer_function(m, L, g):
    num = [0, 0, -3/(2*L)]
    den = [1, 0, (3*g)/(2*L)]

    sys_continuous = signal.TransferFunction(num, den)
    return sys_continuous

# Simulation and plotting function
def simulate_and_plot(reference_amplitude, reference_frequency, show_reference, show_error,
                       noise_amplitude, noise_frequency, show_noise, show_noise_reference,
                       activate_aaf, cutoff_frequency, show_filtered_signal,
                       sampling_frequency, show_sampled_signal, p_gain, i_gain, d_gain, show_controller_output,
                       show_zero_order_hold):
    with output:
        clear_output(wait=True)  # Clear the previous output
        
        # Time array (from 0 to 20 seconds)
        t = np.linspace(0, 20, 1000)

        # Calculate the reference and noise signals
        reference_signal = sinusoidal_signal(t, reference_amplitude, reference_frequency)

        noise_signal = sinusoidal_signal(t, noise_amplitude, noise_frequency)
        
        # Determine the filtered signal (either AAF applied or just noise + reference)
        if activate_aaf:
            combined_signal = reference_signal + noise_signal
            filtered_signal = apply_anti_aliasing_filter(combined_signal, cutoff_frequency)
        else:
            filtered_signal = reference_signal + noise_signal  # When AAF is deactivated, just use the sum

        # Sample the signal
        t_samples = np.arange(0, 20, 1/sampling_frequency)
        indices = np.searchsorted(t, t_samples)
        sampled_signal = filtered_signal[indices]

        # Run discrete PID controller on the sampled signal
        pid_output = pid_controller(sampled_signal, p_gain, i_gain, d_gain, dt=1/sampling_frequency)

        # Calculate zero-order hold signal
        zoh_output = np.zeros_like(t)
        for i, idx in enumerate(indices):
            if i < len(indices) - 1:
                zoh_output[idx:indices[i+1]] = pid_output[i]
            else:
                zoh_output[idx:] = pid_output[i]

        # Calculate plant model response
        sys_continuous = inverted_pendulum_transfer_function(m=1, L=1, g=9.81)
        _, y, _ = signal.lsim(sys_continuous, U=zoh_output, T=t)

        # Feedback loop
        reference_signal = reference_signal - y

        # Plotting the reference and noise signals based on the checkboxes
        fig, ax = plt.subplots(figsize=(8, 6))

        if show_reference:
            ax.plot(t, reference_signal, label=f'Reference (Amplitude={reference_amplitude}, Frequency={reference_frequency} Hz)', color='blue')
        
        if show_noise:
            ax.plot(t, noise_signal, label=f'Noise (Amplitude={noise_amplitude}, Frequency={noise_frequency} Hz)', color='green')
        
        if show_noise_reference:
            ax.plot(t, reference_signal + noise_signal, label=f'Reference + Noise', color='red', linestyle='--')

        # Show filtered signal if checkbox is checked
        if show_filtered_signal:
            ax.plot(t, filtered_signal, label=f'Filtered Signal (Cutoff={cutoff_frequency} Hz)', color='purple', linestyle='-.')

        if show_sampled_signal:
            ax.plot(t_samples, sampled_signal, 'o', label='Sampled Signal', color='black', markersize=2)

        if show_controller_output:
            ax.plot(t_samples, pid_output, 'o', label='PID Controller Output', color='blue', markersize=2)
            
        if show_zero_order_hold:
            ax.plot(t, zoh_output, label='Zero-Order Hold', color='blue')

        ax.set_xlabel('Time (s)')
        ax.set_ylabel('Amplitude (Units)')
        ax.set_title('Reference, Noise, and Filtered Signals')
        ax.set_ylim(-25, 25)  # Set y-axis limits
        ax.grid(True)
        ax.legend()

        plt.tight_layout()
        plt.show()

# Initialize a display object
output = widgets.Output()

# Create sliders for reference signal
reference_amplitude_slider = widgets.FloatSlider(value=5.0, min=0, max=10, step=0.1, description='Amplitude:')
reference_frequency_slider = widgets.FloatSlider(value=0.1, min=0.01, max=1, steps=0.01, description='Frequency (Hz):')
show_reference_checkbox = widgets.Checkbox(value=False, description="Show Reference")

# Show error signal
show_error_checkbox = widgets.Checkbox(value=False, description="Show Error")

# Create sliders for noise signal
noise_amplitude_slider = widgets.FloatSlider(value=1.0, min=0, max=2, step=0.1, description='Amplitude:')
noise_frequency_slider = widgets.FloatSlider(value=4, min=1, max=10, step=0.01, description='Frequency (Hz):')
show_noise_checkbox = widgets.Checkbox(value=False, description="Show Noise")

# Create checkbox for showing both noise and reference together
show_noise_reference_checkbox = widgets.Checkbox(value=True, description="Show Noise + Reference")

# Create section for Anti-Aliasing Filter
activate_aaf_checkbox = widgets.Checkbox(value=True, description="Activate AAF")
cutoff_frequency_slider = widgets.FloatSlider(value=12.0, min=0.1, max=20, step=0.1, description='Cutoff Frequency (Hz):')
show_filtered_signal_checkbox = widgets.Checkbox(value=True, description="Show Filtered Signal")

# Create a section for Sampling
sampling_frequency_slider = widgets.FloatSlider(value=5, min=1, max=10, step=1, description='Sampling Frequency (Hz):')
show_sampled_signal_checkbox = widgets.Checkbox(value=False, description="Show Sampled Signal")

# Create a section for PID gains
pid_p_slider = widgets.FloatSlider(value=1.0, min=0, max=10, step=0.1, description='P Gain:')
pid_i_slider = widgets.FloatSlider(value=0.0, min=0, max=10, step=0.1, description='I Gain:')
pid_d_slider = widgets.FloatSlider(value=0.0, min=0, max=10, step=0.1, description='D Gain:')
show_controller_output_checkbox = widgets.Checkbox(value=False, description="Show Controller Output")

# Create a section for ZOH
show_zero_order_hold_checkbox = widgets.Checkbox(value=False, description="Show Zero-Order Hold")

# Create interactive output for the sliders and checkboxes
widgets.interactive_output(simulate_and_plot, {
    'reference_amplitude': reference_amplitude_slider,
    'reference_frequency': reference_frequency_slider,
    'show_reference': show_reference_checkbox,
    'show_error': show_error_checkbox,
    'noise_amplitude': noise_amplitude_slider,
    'noise_frequency': noise_frequency_slider,
    'show_noise': show_noise_checkbox,
    'show_noise_reference': show_noise_reference_checkbox,
    'activate_aaf': activate_aaf_checkbox,
    'cutoff_frequency': cutoff_frequency_slider,
    'show_filtered_signal': show_filtered_signal_checkbox,
    'sampling_frequency': sampling_frequency_slider,
    'show_sampled_signal': show_sampled_signal_checkbox,
    'p_gain': pid_p_slider,
    'i_gain': pid_i_slider,
    'd_gain': pid_d_slider,
    'show_controller_output': show_controller_output_checkbox,
    'show_zero_order_hold': show_zero_order_hold_checkbox
})

# Arrange widgets into three sections (Reference, Noise, and Anti-Aliasing Filter) with sliders on the right side
reference_section = widgets.VBox([
    show_reference_checkbox, 
    reference_amplitude_slider, 
    reference_frequency_slider
])

error_section = widgets.VBox([
    show_error_checkbox,
])

noise_section = widgets.VBox([
    show_noise_checkbox, 
    noise_amplitude_slider, 
    noise_frequency_slider, 
    show_noise_reference_checkbox
])

aaf_section = widgets.VBox([
    activate_aaf_checkbox, 
    cutoff_frequency_slider,
    show_filtered_signal_checkbox
])

sampling_section = widgets.VBox([
    sampling_frequency_slider,
    show_sampled_signal_checkbox
])

pid_section = widgets.VBox([
    pid_p_slider,
    pid_i_slider,
    pid_d_slider,
    show_controller_output_checkbox,
])

zoh_section = widgets.VBox([
    show_zero_order_hold_checkbox
])

# Create a horizontal layout for the sections and place them next to the plot
control_layout = widgets.HBox([reference_section, noise_section, aaf_section, sampling_section, pid_section, zoh_section])

# Display the controls and the interactive plot
display(widgets.HBox([control_layout]), output)


Output()